In [0]:
dbutils.widgets.text("env","")
env = dbutils.widgets.get("env")

dbutils.widgets.text("raw_schema","")
raw_schema = dbutils.widgets.get("raw_schema")

dbutils.widgets.text("raw_table","")
raw_table = dbutils.widgets.get("raw_table")

dbutils.widgets.text("source_file_name","")
source_file_name = dbutils.widgets.get("source_file_name")

dbutils.widgets.text("container_name","")
container_name = dbutils.widgets.get("container_name")

dbutils.widgets.text("storage_account_name","")
storage_account_name = dbutils.widgets.get("storage_account_name")

dbutils.widgets.text("loadId","")
loadId = dbutils.widgets.get("loadId")

dbutils.widgets.text("aws_last_modified_time","")
aws_last_modified_time = dbutils.widgets.get("aws_last_modified_time")

In [0]:
print(loadId)
print(aws_last_modified_time)

In [0]:
df = spark.sql(f"describe {raw_schema}.{raw_table}").collect()
my_schema = ""
for i in range(len(df)):
    col_name = df[i]['col_name']
    col_type = df[i]['data_type']

    if(col_name == "last_inserted_dttm_azure"):
        break
    my_schema += col_name + " " + col_type + ","
my_schema = my_schema[:-1]

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
File <command-3479366330850910>, line 1
----> 1 df = spark.sql(f"describe {raw_schema}.{raw_table}").collect()
      2 my_schema = ""
      3 for i in range(len(df)):

NameError: name 'raw_schema' is not defined

In [0]:
# reading file from landing container
file_path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{source_file_name}"
landing_df = spark.read.format("csv").option("header", "true").option("mode","permissive").schema(my_schema).load(file_path)

# dropping null records from df
df_without_null = landing_df.na.drop('all')

# removing duplicates
df_without_null_duplicates = df_without_null.dropDuplicates()

source_file_records_count = landing_df.count()

raw_table_file_count = df_without_null_duplicates.count()

# creating the temporary view from table
df_without_null_duplicates.createOrReplaceTempView("df_table")

In [0]:
spark.sql(f"""
    insert overwrite {raw_schema}.{raw_table}
    select *, from_utc_timestamp(current_timestamp(), 'IST') as last_inserted_dttm_azure, '{aws_last_modified_time}' as, '{loadId}'
    from df_table
""")

In [0]:
dbutils.notebook.exit([source_file_records_count,raw_table_file_count])